In [1]:
!pip install git+https://github.com/canesche/nvcc4jupyter.git
!git clone https://github.com/canesche/nvcc4jupyter
%load_ext nvcc_plugin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/canesche/nvcc4jupyter.git to /tmp/pip-req-build-v4ksacnn
  Running command git clone -q https://github.com/canesche/nvcc4jupyter.git /tmp/pip-req-build-v4ksacnn
  Created wheel for ColabPlugin: filename=ColabPlugin-blind-py3-none-any.whl size=12725 sha256=cef7ccbed3f8e7b94e2f344d653689e112898a8a139ca45135024c4f8aef1bd3
  Stored in directory: /tmp/pip-ephem-wheel-cache-tz0e9da_/wheels/8b/84/1d/a2cfacb702e232d99d06524968290563e63fdb70a4c78ee5c4
Failed to build ColabPlugin
    Running setup.py install for ColabPlugin ... done
  DEPRECATION: ColabPlugin was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. A possible replacement is to fix the wheel build issue reported above. You can find discussion regarding this at https://github.com/pypa/pip/issues/8368.
Cloning into 'nvcc4jupyter'...
remote: Enumerating objects: 1


Considere que desejamos gerar um array B através de um array A de mesmo tamanho, de modo que, cada elemento `B[i]` é dado pela soma do elemento `A[i]` mais os `r` elementos anteriores à posição `i` em A mais os `r` elementos posteriores, ou seja:
```
for(j=i-r; j<=i+r; j++)
    B[i]+=A[j]
```
O código dado a seguir já faz o cálculo do array B (armazenado em `bHost`) de forma sequencial. Escreva um kernel CUDA, bem como todo o restante do código necessário para executá-lo, de modo a permitir o cálculo de B de forma paralela.

As posições inválidas no início e final do array A devem ser simplesmente descartadas durante a soma, ou seja, para cálculo de `B[10]` com `r=32`, o resultado será a soma dos elementos de `A[0]` até `A[42]`.

Se atente para os seguinte requisitos:
* O cálculo dos elementos de B deve ser realizado de forma paralela;
* A memória compartilhada da GPU deve ser utilizada de forma a deixar a operação mais eficiente;
* O código deve tratar corretamente arrays grandes divididos em mais de um bloco.

Dicas:
* O código abaixo utiliza a constante `RANGE` para definir o valor de `r` e a constante `BLOCKSIZE` para definir o tamanho de cada bloco do grid;
* Por simplicidade, você pode considerar que o tamanho do array A será sempre múltiplo de `BLOCKSIZE`;
* Considere também que o tamanho de `r` será no máximo igual à metade de `BLOCKSIZE`, ou seja, o array alocado na memória compartilhada terá tamanho igual a `2*BLOCKSIZE`, no máximo.
* A estratégia utilizada pela versão sequencial de testar cada posição (para descobrir se é válida) não é ótima. Muitas comparações desnecessárias são realizadas ao calcular as posições intermediárias. Na versão paralela, é melhor preecher as posições inválidas com o valor 0 (zero) ao copiar os dados para a memória compartilhada.

In [17]:
%%gpu
#include<stdio.h>
#include<cuda.h>
#include<cstring>

#define BLOCKSIZE 1024  //threads por bloco
#define RANGE 512       //deslocamento utilizado para cálculo de B, coloquei para 512

//Luísa de Souza Ferreira - 102026
//Segui a ideia que você tinha me explicado em sala, cada thread irá completar um elemento da primeira metade e depois um elemento da segunda metade
//Você tinha sugerido em colocar o range para 512 também, fiz essas duas coisas e deu certo :)
__global__
void createVecKernel(int* A, int* B, int n) {
     __shared__ int s[2*BLOCKSIZE];
     int tx = threadIdx.x;
     int tid = blockIdx.x * blockDim.x + tx;
     if(tid - RANGE >= 0 )
        s[tx] = A[tid - RANGE];
     else
        s[tx] = 0;

      if(RANGE + tid < n)
        s[BLOCKSIZE+tx] = A[RANGE + tid];
      else
        s[BLOCKSIZE+tx] = 0;

      __syncthreads();
      int sum = 0;
      for(int dist = tx ; dist <= tx + 2 * RANGE; dist++){
          sum += s[dist];
      }

      B[tid] = sum;

}


void createVec(int* h_A, int* h_B, int n) {
 int size = n*sizeof(int);
 int *d_A;
 int *d_B;
 cudaMalloc((void **) &d_A, size);
 cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);
 cudaMalloc((void **) &d_B, size);
 
 createVecKernel<<<ceil(1.0*n/BLOCKSIZE), BLOCKSIZE>>>(d_A, d_B, n);

 cudaMemcpy(h_B, d_B, size, cudaMemcpyDeviceToHost);

 cudaFree(d_A);
 cudaFree(d_B);
}


void initArray(int *a, int n) {
    for (int i =0; i<n; ++i) 
        a[i]=i;
}
void zeraArray(int *b, int n){
    for(int i = 0; i < n; i++)
        b[i] = 0;
}


bool checkArray(int *a, int *b, int n) {
    for (int i =0; i<n; ++i) 
        if (a[i]!=b[i]){
            printf("resultado diferente %d: %d e %d", i, a[i], b[i]);
            return false;
        }
    return true;
}

void printArray(int *a, int n) {
    int maxPrint=20;
    for (int i =0; i<(n>maxPrint ? maxPrint : n); ++i) 
        printf("%3d ",a[i]);
    if (n>maxPrint) printf("... (array truncado)");
    printf("\n");
}

void execHost(int *A, int n, int *B) {
    for(int i=0; i<n; i++) {
        int soma=0;
        for(int j=i-RANGE; j<=i+RANGE; j++)
            if(j >= 0 && j<n)
                soma+=A[j];
        B[i]=soma;
    }
}

int main() {
    int size=1<<20;           //tamanho dos arrays A e B
    int dsize=size*sizeof(int); //tamanho dos dados
    int *a;
    int *b;                     //array B a ser calculado na GPU
    int *bHost;                 //array B calculado na CPU
    a=(int*)malloc(dsize);
    b=(int*)malloc(dsize);
    bHost=(int*)malloc(dsize);
    float elapsed_time = 0;

    initArray(a,size);
    printf("    A: ");
    printArray(a,size);

    createVec(a,b,size);
    printf("B-GPU: ");
    printArray(b,size);
  

    printf("B-CPU: ");
    execHost(a, size, bHost);
    printArray(bHost,size);

    if (checkArray(b,bHost,size))
        printf("Resultados iguais\n");
    else
        printf("Resultados diferentes\n");

    free(bHost);
    free(a);
    free(b);
    return 0;
}

    A:   0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19 ... (array truncado)
B-GPU: 131328 131841 132355 132870 133386 133903 134421 134940 135460 135981 136503 137026 137550 138075 138601 139128 139656 140185 140715 141246 ... (array truncado)
B-CPU: 131328 131841 132355 132870 133386 133903 134421 134940 135460 135981 136503 137026 137550 138075 138601 139128 139656 140185 140715 141246 ... (array truncado)
Resultados iguais

